In [1]:
# pip install selenium

In [2]:
# pip install pandas

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import os
import re

In [4]:
weather_excel_fname = "Weather Data Puller.xlsx"
buffer_load_seconds = 0 # Adjust if missing element error pops out

# Initialisation
weather_site_prefix = "https://lishi.tianqi.com/"
weather_site_suffix = ".html"
# main_dir = os.path.dirname(os.path.realpath(__file__))
main_dir = r"R:\Natural Gas\Gas Data Source\Markets\China"
weather_excel_fpath = f"{main_dir}\\{weather_excel_fname}"
pd.set_option('future.no_silent_downcasting', True)

# Read excel save and format dates
df = pd.read_excel(weather_excel_fpath, header=None)
first_date_row = 3
for row_id in range(first_date_row, len(df[0])):
    if type(date := df.at[row_id, 0]) is not str:
        df.at[row_id, 0] = f"{str(df.at[row_id, 0])[5:7]}/{str(df.at[row_id, 0])[0:4]}"

start_row = df.last_valid_index()
start_url_yr = int(str(df.at[start_row, 0])[3:7])
start_url_mth = int(str(df.at[start_row, 0])[0:2]) + 1
if start_url_mth > 12:
    start_url_yr += 1
    start_url_mth = 1
start_row += 1

end_url_yr = int(time.strftime("%Y", time.localtime()))
end_url_mth = int(time.strftime("%m", time.localtime()))

# Launch chrome
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-error")
options.add_argument("--ignore-ssl-errors")
options.add_argument("log-level=3")
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')  # Disable GPU rendering for speed
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)

# Iterate through year, month, location
for query_yr in range(start_url_yr, end_url_yr+1):
    for query_mth in range(start_url_mth, 13): # Set the month to stop at (e.g 3 means stop at February)
        if query_yr == end_url_yr and query_mth > end_url_mth:
            break

        df.at[start_row, 0] = f"{query_mth:02d}/{query_yr}"
        query_url_date = f"{query_yr}{query_mth:02d}"

        for location_url_name in df.loc[[1]].values[0]:
            if type(location_url_name) is str:
                query_url = f"{weather_site_prefix}{location_url_name}/{query_url_date}{weather_site_suffix}"

                # Access weather site
                driver.get(query_url)
                time.sleep(buffer_load_seconds)
                if driver.current_url == 'https://lishi.tianqi.com/404.html':
                    continue

                # Locate the temperature elements (adjust the XPath if necessary)
                high_temp_value = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[3]/ul/li[1]/div[1]").text
                low_temp_value = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[3]/ul/li[1]/div[2]").text

                # Extract only the numeric part from the string (including negative sign if necessary)
                high_temp_value = re.search(r'-?\d+', high_temp_value).group()
                low_temp_value = re.search(r'-?\d+', low_temp_value).group()
                ave_temp_value = (float(high_temp_value) + float(low_temp_value)) / 2

                # Update df
                # replace_row = df[0].eq(f"{query_mth:02d}/{query_yr}").idxmax()
                replace_col = df.loc[1].eq(location_url_name).idxmax()
                df.at[start_row, replace_col] = high_temp_value
                df.at[start_row, replace_col+1] = low_temp_value
                df.at[start_row, replace_col+2] = ave_temp_value

            # else: continue
            # break

        # print(df)
        # # break

        start_row += 1
        df.to_excel(weather_excel_fpath, sheet_name="Puller", header=False, index=False)

    start_url_mth = 1
    # break

# print(df)

# Close the browser after scraping
driver.quit()

# Open excel
os.startfile(r"R:\Natural Gas\Gas Data Source\Markets\China\Weather Data Puller.xlsx")